We use following tools

1. Python 3
2. Jupyter Notebook
3. Azure CLI

# Model Training

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50,InceptionResNetV2
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.applications.inception_resnet_v2 import preprocess_input
from keras import optimizers
import matplotlib.pyplot as plt

In [0]:
PATH = 'rustdata2'
sz = 224
batch_size = 4
train_data_dir = f'{PATH}train' 
validation_data_dir = f'{PATH}valid'

#data augmentations for training data
train_datagen = ImageDataGenerator(rescale=1. / 255,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True,vertical_flip=True,rotation_range=90)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='categorical')


In [0]:
# using InceptionResNetV2 as base model pre trained on imagenet
base_model = InceptionResNetV2(weights = 'imagenet',include_top=False,input_shape=(sz,sz,3))

In [0]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4, activation='softmax'))

In [0]:
for layer in base_model.layers[:500]: 
  layer.trainable = False
for layer in base_model.layers[500:]: 
  layer.trainable = True
model.compile(optimizer=optimizers.SGD(lr=0.001), loss='categorical_crossentropy'
              , metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator, train_generator.n // batch_size
, epochs=3,workers=4,validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

# Model Deployment

### Initialize Workspace

In [0]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

ws = Workspace(resource_group='DemoGroup',workspace_name='demospace',subscription_id='5959c85b-cacb-4c20-b1fb-e47c40ed5aff')

### Save the model 

In [0]:
model.save('kerasmodel.pkl')

### Register a model

In [0]:
from azureml.core.model import Model
model_name = "kerasmodel"
model_path = "./kerasmodel.pkl"

model = Model.register(
    model_path=model_path,
    model_name=model_name,
    tags={"data": "rust", "model": "classification"},
    description="Classifying rust and non rust images",
    workspace=ws
)

## Deploy as a webservice

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI.

To build the correct environment for ACI, provide the following:

1. A scoring script to show how to use the model
2. An environment file to show what packages need to be installed
3. A configuration file to build the ACI
4. The model you trained before

### Write a scoring script 
In order to create a web service, you will create a scoring script that will load the models, perform the prediction, and return the result. Azure ML uses init() and run() functions inside this scoring script for that purpose. The init() function initializes the web service and loads the saved model. The run() function uses the model and the input data to return a prediction which is executed on a scoring call. The init() function should not have any errors otherwise the deployment will fail.

In [0]:
%%writefile script.py

import numpy as np
import os
import sys
import keras as K
import json
import base64
from io import BytesIO
from PIL import Image
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input

def init():
    
    global model  

    print("Executing init() method...")
    print("Python version: " + str(sys.version) + ", keras version: " + K.__version__)
    # Load the model 
    #'K.models.load_model('azureml-models/model_name/model_version/model_path')
    model = K.models.load_model('azureml-models/kerasmodel/1/kerasmodel.pkl')
    
    return

def predict(model, img):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict_classes(x)
    y = model.predict(x)[0]
    y = np.expand_dims(y, axis=-1)
    b = []
    a = y[0]
    b.append(np.float(a))
    a = y[1]
    b.append(np.float(a))
    b.append(preds[0])
    return b

def run(inputString):
    
    responses = []
    #load the JSON data recieved
    base64Dict = json.loads(inputString)
    
    #the JSON data here was base64 encoded
    #so it is decoded first
    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    
    #load image
    img = image.load_img(img_buffer,target_size=(224, 224))
    
    #make prediction to model
    preds = predict(model, img)
    LABELS = ["Non Rust", "Rust"]
    resp = LABELS[preds[-1]]
    responses.append(preds[0])
    responses.append(preds[1])
    responses.append(resp)
    
    #return JSON response
    return json.dumps(responses)
    
  
if __name__ == "__main__":
    init()
    # input data
    img_path = '00000668.JPG'
    encoded = None
    # data is encoded in base64 
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    resp = run(body)
    print(resp)

### Create environment file
Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. See the myenv.yml file included, it contains all the dependencies required for this project, you can include further more as well accroding to your needs.

In [0]:
import os
with open(os.path.join("myenv.yml"),"r") as f:
    print(f.read())

### Create ACI configuration
Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [0]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2, 
    memory_gb=2, 
    tags={"data": "rust",  "method" : "keras"}, 
    description='Predict rust with keras'
)

### Create Docker image
Estimated time to complete: about 3-5 minutes

Build an image and register that image under the workspace using:

1. The scoring file (script.py)
2. The environment file (myenv.yml)
3. The model files

In [0]:
%%time
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(
    execution_script="script.py", 
    runtime="python", 
    conda_file="myenv.yml"
)

image = ContainerImage.create(
    name = "keras-rust-img",
    models = [model],
    image_config = image_config,
    workspace = ws
)

image.wait_for_creation(show_output = True)

In [0]:
print(image.image_location)

### Deploy the image in ACI
Estimated time to complete: about 3-5 minutes

The following code goes through these steps:

1. Send the image to the ACI container.
2. Start up a container in ACI using the image.
3. Get the web service HTTP endpoint.

Creating and deploying the image, could be done with following:

In [0]:
%%time
from azureml.core.webservice import Webservice

aci_service = Webservice.deploy_from_image(
    workspace = ws, 
    name = 'keras-rust-aci-svc',
    image = image,
    deployment_config = aciconfig
)

aci_service.wait_for_deployment(show_output=True)

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [0]:
print(aci_service.scoring_uri)

### Debugging
If the deployment fails it is usually due to errors in scoring python script. You can see the logs by using " aci_service.get_logs() " or visit https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-troubleshoot-deployment

In [0]:
aci_service.get_logs()

You can even refer this github repo https://github.com/SaschaDittmann/TensorFlow101